In [2]:
%matplotlib inline
import os, io, json, urllib, numpy as np, codecs
import glob, pandas as pd
pd.options.display.max_rows = 999

In [3]:
def h_(citation):
    citation=list(citation)
    h=0
    citation.sort(reverse=True)
    for i in range(1,len(citation)+1):
        if citation[i-1] >= i:
            h=i
    return h

In [188]:
# skip if result.txt exists
y17 = glob.glob('data/August/2017/*.txt')
y16 = glob.glob('data/August/2016/*.txt')
y15 = glob.glob('data/August/2015/*.txt')
y14 = glob.glob('data/August/2014/*.txt')
y14_17=y14+y15+y16+y17
with open("data/August/data.txt", "wb") as outfile:
    for f in y14_17:
        with open(f, "rb") as infile:
            lines=infile.read().split('\n')
            for line in lines:
                if len(line.split('\t'))==10:
                    outfile.write(line+'\n')

NameError: name 'data_path' is not defined

In [173]:
df=pd.read_table(data_path+'/data.txt',header=None)
df.columns=['doi','year','ref','citation','issnp','issne','journal','publisher','author','lic']
df.shape

(11316994, 10)

In [174]:
df['yy']=df.year.str[1:5]
df['mm']=df.year.apply(lambda x: int(x.replace(']','').split(', ')[1]) if len(x.split(', '))>1 else 0)

In [175]:
df=df[~((df.yy=='2014') & (df.mm<9))]
df.shape

(9293675, 12)

In [177]:
df['year']=df['yy'].astype(int)

In [178]:
df=df[(df['year']<=2017) & (df['year']>=2014)]
df.shape

(9219121, 12)

In [179]:
df['count']=1
df['cited']=df['citation'].apply(lambda x: 1 if x>0 else 0)
df['un-cited']=df['citation'].apply(lambda x: 0 if x>0 else 1)

In [180]:
df=df[(~df.issnp.isnull()) | (~df.issne.isnull())]
df.shape

(9210602, 15)

In [181]:
#df=df[['doi','year','mm','citation','issnp','issne']].fillna('_')
df=df.fillna('_')
df['issn']=df.issnp+','+df.issne
df['issn']=df.issn.str.replace(',_','')
df['issn']=df.issn.str.replace('_,','')

In [182]:
df['issn']=df.issn.apply(lambda x:x[0:9] if x[10:19]==x[0:9] else x)

In [183]:
s=df.issn.str.split(',',expand=True).stack()

In [184]:
i = s.index.get_level_values(0)

In [185]:
df2 = df.loc[i].copy()

In [186]:
df2["@id"] = s.values

In [24]:
import json

with open('journals.json') as data_file:    
    id_subject_dic = json.load(data_file)

In [25]:
df2['subjects']=df2['@id'].apply(lambda x: ';'.join(id_subject_dic[x]['e:subjects']) if x in id_subject_dic else '')

In [88]:
unique_subjects=[]
all_subjects=df2['subjects'].values
for journal_subjects in all_subjects:
    subjects=journal_subjects.split(';')
    for subject in subjects:
        if subject not in unique_subjects: unique_subjects.append(subject)

In [89]:
len(unique_subjects)

329

In [170]:
for subject in unique_subjects[0:1]:
    subject=''
    print subject
    if subject=='':
        df_subject=df2.sort_values('citation', ascending= False)[:20]
    else:
        df_subject=df2[df2.subjects.apply(lambda x: subject in x.split(';'))].sort_values('citation', ascending= False)[:20]
    top_dois=df_subject[['doi','@id','citation']].to_dict('list')
    zips=zip(top_dois['doi'],top_dois['@id'],top_dois['citation'])
    items=[];
    for doi,issn,citation in zips:
        item={}; item_cleaned={}
        url='http://api.crossref.org/works/'+doi
        response = urllib.urlopen(url).read()    
        json_result=json.loads(response)
        item = json_result['message']
        author='0';year_print=[9999,99,99];year_online=[9999,99,99];
        if 'title' in item:
            title=item['title'][0]
        if 'published-print' in item:
            year_print=item['published-print']['date-parts'][0]                
        if 'published-online' in item:
            year_online=item['published-online']['date-parts'][0]
        year=min(year_print,year_online)
        if 'DOI' in item: doi=item['DOI']
        if 'container-title' in item: journal=','.join(item['container-title'])
        if 'publisher' in item: pub=item['publisher']; pub=''if pub==None else pub
        if 'references-count' in item: ref=str(item['references-count'])
        if 'author' in item: authors=item['author']
        if len(year)>1 :
            month=str(year[1]) 
        else: month=''
        item_cleaned={'doi':doi,
                      'title':title,
                      'year':str(year[0])+'-'+month,
                      'cite':str(citation),
                      'issn':issn, 
                      'journal':journal,
                      'pub':pub,
                      'authors':authors}
        items.append(item_cleaned)  
    with open('sjdb/top_'+subject+'.json', 'w') as fp:
        json.dump(items, fp)    

In [168]:
for subject in unique_subjects:
    df_lic=df2[df2.subjects.apply(lambda x: subject in x.split(';'))]
    top_lics=df_lic[['lic','count']].groupby('lic').count().sort_values('count',ascending=False).head().reset_index().to_dict('list')
    zips=zip(top_lics['lic'],top_lics['count'])
    total_articles=str(df_lic.shape[0])
    items=[{'count':total_articles,'lic':'all'}];
    for lic,count in zips:
        item={}; 
        item={'lic':lic,'count':str(count)}
        items.append(item)  
    with open('sjdb/lic_'+subject+'.json', 'w') as fp:
        json.dump(items, fp)    